In [ ]:
!pip install torch torchvision transformers pillow groq -q
import os
import torch
import cv2
import numpy as np
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification
from google.colab import files
from groq import Groq

from google.colab import userdata
os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError(" GROQ_API_KEY not found. Please set it in Colab secrets.")

client = Groq(api_key=GROQ_API_KEY)

model_name = "Ateeqq/ai-vs-human-image-detector"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)
model.eval()

print(" Upload your video file (.mp4, .mov, etc.)")
uploaded = files.upload()
video_path = next(iter(uploaded.keys()))

def extract_frames(video_path, frame_rate=1):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        raise ValueError(" Could not read FPS from video.")
    interval = int(fps * frame_rate)
    frames = []
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(frame_rgb))
        count += 1

    cap.release()
    return frames

print("Extracting frames...")
frames = extract_frames(video_path, frame_rate=1)
print(f" Extracted {len(frames)} frames.")

ai_probs = []
human_probs = []

for img in frames:
    inputs = processor(images=img, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)[0]
        ai_probs.append(probs[0].item())
        human_probs.append(probs[1].item())

avg_ai = np.mean(ai_probs)
avg_human = np.mean(human_probs)
diff = abs(avg_ai - avg_human)
if diff >= 0.7:
    confidence = "High"
elif diff >= 0.3:
    confidence = "Medium"
else:
    confidence = "Low"

result_label = "AI-generated" if avg_ai > avg_human else "Human-generated"

prompt = f"""
You are an AI video analysis expert.
The detector processed {len(frames)} frames from a video.
Average probabilities:
- AI-generated: {avg_ai:.4f}
- Human-generated: {avg_human:.4f}
Confidence: {confidence}

Give a short, human-readable explanation (1–2 sentences) of why the video was likely classified as {result_label}.
Mention aspects like motion smoothness, lighting consistency, texture, or anomalies if relevant.
"""

response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.6,
)

explanation = response.choices[0].message.content.strip()

print("\n Video Detection Summary:")
print(f" AI Probability: {avg_ai:.4f}")
print(f" Human Probability: {avg_human:.4f}")
print(f" Confidence: {confidence}")
print(f" Explanation: {explanation}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/372M [00:00<?, ?B/s]

 Upload your video file (.mp4, .mov, etc.)


KeyboardInterrupt: 